In [2]:
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from PIL import Image

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)

#===================MAIN SCRIPTE=====================
root_dir = '/glade/u/home/hongli/work/sharp/basins/scripts'
os.chdir(root_dir)
output_folder='step5_GMET_NLDAS_forcing_hydro'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
outputfile = 'GMET_NLDAS_forcing_hydro.png'

gmet_forcing_file = '/glade/u/home/hongli/work/sharp/basins/tayprk_calib_template/model/forcings/gmet_3hr_summa/ens_forc.UPCO.huc12.001_20100101-20101231.nc'
nldas_forcing_file = '/glade/u/home/hongli/work/sharp/basins/tayprk_huc12_nldas/forcings/forcing_nldas.tayprk.2010.nc'
gmet_sim_file='/glade/u/home/hongli/work/sharp/basins/tayprk_huc12_metsim/route/output/all.nc'
nldas_sim_file='/glade/u/home/hongli/work/sharp/basins/tayprk_huc12_nldas/route/output/all.nc'
obs_file='Taylor_park_09107000.txt'
gmet_calib_sim_file = '/glade/u/home/hongli/work/sharp/basins/tayprk_huc12_metsim_calib/route/output/all.nc'
# gmet_calib_sim_file = '/glade/u/home/hongli/work/sharp/basins/scripts/all_hydro-c1_calib1.nc'

forcing_vnames = ['pptrate', 'airtemp', 'SWRadAtm']
forcing_vunits = ['kg m-2 s-1', 'K', '$W/m^2$']
t_vname = 'time'

hru_indexs = [0, 1, 2]
force_time_format='%Y-%m-%d %H'
time_step = 3 #hr
time_num_perday = 24/time_step

force_plot_start_str='2010-01-02 00'
force_plot_end_str='2010-12-31 23'
formatter = DateFormatter('%m/%d/%y')
alpha = 0.5
dpi_value = 100
ylabels = ['Precip Rate', 'Air Temprature', 'SW Radiation']

# read forcing data
f_gmet = xr.open_dataset(gmet_forcing_file)
force_time = f_gmet[t_vname].dt.strftime(force_time_format).values[:]
force_start_str=force_time[0]
force_end_str=force_time[-1]
f_nldas = xr.open_dataset(nldas_forcing_file)

# read flow data
q_vname2 = 'IRFroutedRunoff'
t_vname = 'time'
sim_time_format = '%Y-%m-%d'
sim_plot_start_str = '2010-01-02'
sim_plot_end_str = '2010-12-31'

f_nldas_sim = xr.open_dataset(nldas_sim_file)
nldas_sim_irf = f_nldas_sim[q_vname2].values[:]
nldas_sim_time = f_nldas_sim[t_vname].dt.strftime(sim_time_format).values[:]
f_nldas_sim.close()

sim_start_str = nldas_sim_time[0]
sim_end_str = nldas_sim_time[-1]

f_gemt_sim = xr.open_dataset(gmet_sim_file)
gmet_sim_irf = f_gemt_sim[q_vname2].values[:]
f_gemt_sim.close()

# read observed flow (f^3/s)
obs = []
obs_time = []
with open(os.path.join(root_dir, obs_file)) as f:
    for line in f:
        line = line.strip()
        if line and line.startswith('USGS'):
            splits = line.split()
            obs_time.append(splits[2])
            if len(splits)>3:#flow data exists
                obs.append(float(splits[3])/35.315) #convert to m^3/s
            else:
                obs.append(np.nan)
obs_start=(datetime.datetime.strptime(sim_start_str,sim_time_format) - datetime.datetime.strptime(obs_time[0],sim_time_format)).days
obs_len=(datetime.datetime.strptime(sim_end_str,sim_time_format) - datetime.datetime.strptime(sim_start_str,sim_time_format)).days+1
obs_useful = obs[obs_start:obs_start+obs_len]

# read calibrated sim flow
f = xr.open_dataset(gmet_calib_sim_file)
gmet_sim_irf_calib = f[q_vname2].values[:]
gmet_sim_time_calib = f[t_vname].dt.strftime(sim_time_format).values[:]
f.close()

gmet_calib_start=(datetime.datetime.strptime(sim_start_str,sim_time_format) - datetime.datetime.strptime(gmet_sim_time_calib[0],sim_time_format)).days
gmet_len=(datetime.datetime.strptime(sim_end_str,sim_time_format) - datetime.datetime.strptime(sim_start_str,sim_time_format)).days+1
gmet_sim_irf_calib = gmet_sim_irf_calib[gmet_calib_start:gmet_calib_start+gmet_len]

# plot
os.chdir(os.path.join(root_dir, output_folder))
for hru_index in hru_indexs:
    print('hru '+str(hru_index+1))
    hru_fig_file = os.path.join(root_dir, output_folder, 
                               outputfile.split('.')[0]+str(hru_index+1)+'.'+outputfile.split('.')[1])

    row_num=len(forcing_vnames)+1 # forcings + hydrograph 
    fig, ax = plt.subplots(row_num,1)
    fig.set_figwidth(6.5) #190mm
    fig.set_figheight(6.5*0.4*row_num)

    plot_var_count = 0        
    for k in range(row_num):
        plot_alpha = chr(ord('a')+k)

        # forcings
        if k < len(forcing_vnames):

            var_index = k
            var_name = forcing_vnames[var_index]

            plot_start=int(((datetime.datetime.strptime(force_plot_start_str,force_time_format)-datetime.datetime.strptime(force_start_str,force_time_format)).days)*time_num_perday)
            plot_len=int(((datetime.datetime.strptime(force_plot_end_str,force_time_format)-datetime.datetime.strptime(force_plot_start_str,force_time_format)).days)*time_num_perday)

            plot_start_date=datetime.datetime.strptime(force_plot_start_str,force_time_format)
            t=[plot_start_date+datetime.timedelta(hours=hr*time_step) for hr in range(0,plot_len)]
            t_plot=[mpl.dates.date2num(hr) for hr in t[0:plot_len]]  

            gmet_var_value = f_gmet[var_name].values[:]
            nldas_var_value = f_nldas[var_name].values[:]
            var_unit = forcing_vunits[forcing_vnames.index(var_name)]

            if var_unit == 'kg m-2 s-1':
                var_unit = 'mm/d'
                gmet_var_value = gmet_var_value*86400 # flux from kg/m^2/s to mm/day
                nldas_var_value = nldas_var_value*86400
            elif var_unit == 'K':
                var_unit = '$^\circ$C'
                gmet_var_value = gmet_var_value - 273.15 # temp from K to C
                nldas_var_value = nldas_var_value - 273.15
#             elif var_unit == 'W m-2':
#                 var_unit = 'mm/d'
#                 gmet_var_value = gmet_var_value/344000*12*3600 # W/m2 -> mm/d SWE
#                 nldas_var_value = nldas_var_value/344000*12*3600

            gmet_var_value_plot = gmet_var_value[plot_start+0:plot_start+plot_len, hru_index] 
            nldas_var_value_plot = nldas_var_value[plot_start+0:plot_start+plot_len, hru_index] 
            ax[k].plot_date(t_plot, gmet_var_value_plot, 'bo-', linewidth=1.0, markersize=0.0, alpha=alpha, label='GMET')   
            ax[k].plot_date(t_plot, nldas_var_value_plot, 'r^-', linewidth=0.75, markersize=0.0, alpha=alpha, label='NLDAS')   

            ylabel = ylabels[k]+' ('+var_unit+')'
            ax[k].set_ylabel(ylabel, fontsize='small', horizontalalignment='center')
            ax[k].yaxis.set_tick_params(labelsize='small')

            ax[k].xaxis.set_major_formatter(formatter)
            ax[k].xaxis.set_tick_params(labelsize='small')#rotation=30, 

            title_str = '('+ chr(ord('a')+k) +') ' + ylabels[k]
            ax[k].set_title(title_str, fontsize='small', fontweight='bold')
            ax[k].legend(loc='best', fontsize='x-small', framealpha=0.5) 

            del gmet_var_value, gmet_var_value_plot, nldas_var_value, nldas_var_value_plot

            # flow
        else:
            plot_start=((datetime.datetime.strptime(sim_plot_start_str,sim_time_format)-datetime.datetime.strptime(sim_start_str,sim_time_format)).days)
            plot_len=((datetime.datetime.strptime(sim_plot_end_str,sim_time_format)-datetime.datetime.strptime(sim_plot_start_str,sim_time_format)).days+1)

            plot_start_date=datetime.datetime.strptime(sim_plot_start_str,sim_time_format)
            t=[plot_start_date+datetime.timedelta(days=d) for d in range(0,plot_len)]
            t_plot=[mpl.dates.date2num(d) for d in t[0:plot_len]]        

            obs_plot=obs_useful[plot_start+0:plot_start+plot_len]
            nldas_sim_irf_plot=nldas_sim_irf[plot_start+0:plot_start+plot_len]
            gmet_sim_irf_plot=gmet_sim_irf[plot_start+0:plot_start+plot_len]
            gmet_sim_irf_calib_plot=gmet_sim_irf_calib[plot_start+0:plot_start+plot_len]

            ax[k].plot_date(t_plot, gmet_sim_irf_plot, 'b-o', linewidth=1.0, markersize=0.0, alpha=alpha, label='GMET') 
            ax[k].plot_date(t_plot, gmet_sim_irf_calib_plot, color='green', marker='o', linestyle='dashed',
                            linewidth=1.0, markersize=0.0, alpha=alpha, label='GMET (calibrate)') 
            ax[k].plot_date(t_plot, nldas_sim_irf_plot, 'r-^', linewidth=1.0, markersize=0.0, alpha=alpha, label='NLDAS') 
            ax[k].plot_date(t_plot, obs_plot, 'k-s', linewidth=1.0, markersize=0.0, alpha=alpha, label='Observation')   

            title_str = '('+ chr(ord('a')+k) +') ' + 'Hydrograph'
            ax[k].set_title(title_str, fontsize='small', weight='semibold')
            ax[k].set_xlabel('Date', fontsize='small')
            ax[k].set_ylabel('Flow $(m^3/s)$', fontsize='small')
            ax[k].xaxis.set_major_formatter(formatter)
            ax[k].xaxis.set_tick_params(labelsize='small')
            ax[k].yaxis.set_tick_params(labelsize='small')
            ax[k].legend(loc='best', fontsize='x-small', framealpha=0.5) 

    fig.tight_layout()
    fig.savefig(hru_fig_file, dpi=dpi_value)
    plt.close(fig)                     

f_gmet.close()
f_nldas.close()
os.chdir(root_dir)
print('Done')



hru 1
hru 2
hru 3
Done


In [3]:
gmet_sim_irf_calib

array([[3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],
       [3.31292000e-07],


In [19]:
gmet_var_value = f[var_name].values[:]

TypeError: '_io.TextIOWrapper' object is not subscriptable

In [22]:
force_time[8]

'2010-01-02 00'